## Creating an Explicit Latent Matrix Factorization Recommender System for Books10k Dataset

In [30]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix


First we load the data set:






In [36]:
ratingsDataMatrix = pd.read_csv('/home/robot-tumas/Desktop/projects/python/recSysTutorial/datasets/goodBooks10k/ratings.csv',engine='python')

In [37]:
#we should  drop duplicates to clean the data:
ratingsDataMatrix = ratingsDataMatrix.drop_duplicates()
#lets look at some of the ratings:
print('we have ',ratingsDataMatrix.shape[0], 'observations and ', ratingsDataMatrix.shape[1], 'columns')
print('the columns are:', ratingsDataMatrix.columns)
print('the number of unique users we have is:', len(ratingsDataMatrix.user_id.unique()))
print('the number of unique books we have is:', len(ratingsDataMatrix.book_id.unique()))



we have  980112 observations and  3 columns
the columns are: Index(['book_id', 'user_id', 'rating'], dtype='object')
the number of unique users we have is: 53424
the number of unique books we have is: 10000


In [61]:
#from scipy we can get a set up a sparse coordinate matrix with the ratings and users. 
ratingsMatrix = coo_matrix((ratingsDataMatrix['rating'].astype(float),
                     (ratingsDataMatrix['book_id'], ratingsDataMatrix['user_id'])))

## we are now ready for FunkSVD. This is where we do singular value decomposition and set up our factors for R= P * Q

In [70]:
#first, its always a good idea to mean center your data. 
#This will usually normalize the data and allow for a better distribution to work with. 
#numpy is great for this.
meanMatrix = ratingsMatrix.mean(axis=1)
print(meanMatrix.shape) # this means we gotta transpose it when we subtract.
ratingCenteredMatrix = ratingsMatrix - meanMatrix.reshape(-1,1)
ratingCenteredMatrix

(10001, 1)


In [ ]:
#lets do SVD: with 30 factors
from scipy.sparse.linalg import svds
U, sigma, V = svds(ratingCenteredMatrix,k=30)
del(ratingCenteredMatrix)# this really helps free up active memory
sigma = np.diag(sigma)# make sigma really Sigma
print(sigma)#lets see the diagonal values

P = np.dot(U,sigma)
QT = V